# In-hospital Mortality Prediction

This notebook showcases in-hospital mortality prediction due to heart failure on a subset of the MIMIC-III dataset using CyclOps.

## Import Libraries

In [ ]:
import copy

import plotly.express as px
import plotly.graph_objects as go
from datasets import Dataset
from datasets.features import ClassLabel
from kaggle.api.kaggle_api_extended import KaggleApi
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from cyclops.data.slicer import SliceSpec
from cyclops.evaluate.fairness import FairnessConfig  # noqa: E402
from cyclops.evaluate.metrics import MetricCollection, create_metric
from cyclops.models.catalog import create_model
from cyclops.process.feature.feature import TabularFeatures
from cyclops.report import ModelCardReport
from cyclops.report.plot.classification import ClassificationPlotter
from cyclops.tasks.mortality_prediction import MortalityPredictionTask
from cyclops.utils.file import join, load_dataframe

CyclOps offers a package for documentation of the model through a model card. The `ModelCardReport` class is used to populate and generate the model card as an HTML file. The model card has the following sections:
- Model Details: This section contains descriptive metadata about the model such as the owners, version, license, etc.
- Model Parameters: This section contains the technical details of the model such as the model architecture, training parameters, etc.
- Considerations: This section contains descriptions of the considerations involved in developing and using the model such as the intended use, limitations, etc.
- Quantitative Analysis: This section contains the performance metrics of the model for different sets of the data and subpopulations.
- Explainaibility Analysis: This section contains the explainability metrics of the model.
- Fairness Analysis: This section contains the fairness metrics of the model.

We will use this to document the model development process as we go along and generate the model card at the end.

```{note}
The model card tool is a work in progress and is subject to change.
```

In [ ]:
report = ModelCardReport()

## Constants

In [ ]:
DATA_DIR = "./data"
RANDOM_SEED = 85
NAN_THRESHOLD = 0.75
TRAIN_SIZE = 0.8

## Data Loading

Before starting, make sure to install the Kaggle API by running `pip install kaggle`. To use the Kaggle API, you need to sign up for a Kaggle account at https://www.kaggle.com. Then go to the 'Account' tab of your user profile (`https://www.kaggle.com/<username>/account`) and select 'Create API Token'. This will trigger the download of kaggle.json, a file containing your API credentials. Place this file in the location `~/.kaggle/kaggle.json` on your machine.

In [ ]:
api = KaggleApi()
api.authenticate()
api.dataset_download_files(
    "saurabhshahane/in-hospital-mortality-prediction", path=DATA_DIR, unzip=True
)

In [ ]:
df = load_dataframe(join(DATA_DIR, "data01.csv"), file_format="csv")
df

## Data Inspection and Preprocessing

#### Drop NaNs based on the `NAN_THRESHOLD`

In [ ]:
null_counts = df.isnull().sum()[df.isnull().sum() > 0]
fig = go.Figure(data=[go.Bar(x=null_counts.index, y=null_counts.values)])

fig.update_layout(
    title="Number of Null Values per Column",
    xaxis_title="Columns",
    yaxis_title="Number of Null Values",
    height=600,
)

fig.show()

In [ ]:
thresh_nan = int(NAN_THRESHOLD * len(df))
df = df.dropna(axis=1, thresh=thresh_nan)
df = df.dropna(axis=0, subset=["outcome"])

#### Gender values

In [ ]:
# Female: gender = 1
# Male: gender = 0
df = df.rename(columns={"gendera": "gender"})
df["gender"] = df["gender"].replace({1: 0, 2: 1})

In [ ]:
fig = px.pie(df, names="gender")

fig.update_layout(
    title="Gender Distribution",
)

fig.show()

####  Age distribution

In [ ]:
fig = px.histogram(df, x="age")
fig.update_layout(
    title="Age Distribution",
    xaxis_title="Age",
    yaxis_title="Count",
    bargap=0.2,
)

fig.show()

#### Outcome distribution

In [ ]:
df["outcome"] = df["outcome"].astype("int")

In [ ]:
fig = px.pie(df, names="outcome")
fig.update_traces(textinfo="percent+label")
fig.update_layout(title_text="Outcome Distribution")
fig.update_traces(
    hovertemplate="Outcome: %{label}<br>Count: %{value}<br>Percent: %{percent}"
)
fig.show()

In [ ]:
class_counts = df["outcome"].value_counts()
class_ratio = class_counts[0] / class_counts[1]
class_ratio

From all the features in the dataset, we select 20 of them which was reported by [Li et al.](https://pubmed.ncbi.nlm.nih.gov/34301649/)  to be the most important features in this classification task. 

In [ ]:
features_list = [
    "Anion gap",
    "Lactic acid",
    "Blood calcium",
    "Lymphocyte",
    "Leucocyte",
    "heart rate",
    "Blood sodium",
    "Urine output",
    "Platelets",
    "Urea nitrogen",
    "age",
    "MCH",
    "RBC",
    "Creatine kinase",
    "PCO2",
    "Blood potassium",
    "Diastolic blood pressure",
    "Respiratory rate",
    "Renal failure",
    "NT-proBNP",
]
features_list = sorted(features_list)

#### Identifying feature types

Cyclops `TabularFeatures` class helps to identify feature types, an essential step before preprocessing the data. Understanding feature types (numerical/categorical/binary) allows us to apply appropriate preprocessing steps for each type.

In [ ]:
tab_features = TabularFeatures(
    data=df.reset_index(),
    features=features_list,
    by="ID",
    targets="outcome",
)
tab_features.types

#### Creating data preprocessors

We create a data preprocessor using sklearn's ColumnTransformer. This helps in applying different preprocessing steps to different columns in the dataframe. For instance, binary features might be processed differently from numeric features.

In [ ]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())]
)

binary_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent"))]
)

In [ ]:
numeric_features = sorted((tab_features.features_by_type("numeric")))
numeric_indices = [
    df[features_list].columns.get_loc(column) for column in numeric_features
]
numeric_features

In [ ]:
binary_features = sorted(tab_features.features_by_type("binary"))
binary_features.remove("outcome")
binary_indices = [
    df[features_list].columns.get_loc(column) for column in binary_features
]
binary_features

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_indices),
        ("bin", binary_transformer, binary_indices),
    ],
    remainder="passthrough",
)

Let's document the dataset in the model card. This can be done using the `log_dataset` method, which takes the following arguments:
- description: A description of the dataset.
- citation: The citation for the dataset.
- link: A link to a resource for the dataset.
- license_id: The SPDX license identifier for the dataset.
- version: The version of the dataset.
- features: A list of features in the dataset.
- split: The split of the dataset (train, test, validation, etc.).
- sensitive_features: A list of sensitive features used to train/evaluate the model.
- sensitive_feature_justification: A justification for the sensitive features used to train/evaluate the model.

In [ ]:
report.log_dataset(
    description="MIMIC-III is a large, freely-available database comprising of \
        deidentified health-related data associated with over forty thousand \
        patients who stayed in the ICU of the Beth Israel Deaconess Medical Center \
        between 2001 and 2012. The data includes vital sign measurements, medications, \
        laboratory measurements, imaging reports, and more.",
    citation="Zhou, Jingmin et al. (2021), Prediction model of in-hospital mortality \
        in intensive care unit patients with heart failure: machine learning-based, \
        retrospective analysis of the MIMIC-III database, Dryad, Dataset, \
        https://doi.org/10.5061/dryad.0p2ngf1zd",
    link="""
    https://www.kaggle.com/datasets/saurabhshahane/in-hospital-mortality-prediction
    """,
    license_id="CC0-1.0",
    version="Version 1",
    features=features_list,
    sensitive_attributes=["gender", "age"],
    sensitive_feature_justification="Demographic information like age and gender \
        often have a strong correlation with health outcomes. For example, older \
        patients are more likely to have a higher risk of mortality.",
)

## Creating Hugging Face Dataset

We convert our processed Pandas dataframe into a Hugging Face dataset, a powerful and easy-to-use data format which is also compatible with CyclOps models and evaluator modules. The dataset is then split to train and test sets.

In [ ]:
dataset = Dataset.from_pandas(df)
dataset.cleanup_cache_files()
dataset

In [ ]:
dataset = dataset.cast_column("outcome", ClassLabel(num_classes=2))
dataset = dataset.train_test_split(
    train_size=TRAIN_SIZE, stratify_by_column="outcome", seed=RANDOM_SEED
)

## Model Creation

CyclOps model registry allows for straightforward creation and selection of models. This registry maintains a list of pre-configured models, which can be instantiated with a single line of code. Here we use a SGD classifier to fit a logisitic regression model. The model configurations can be passed to `create_model` based on the sllearn parameters for SGDClassifer.

In [ ]:
model_name = "sgd_classifier"
model = create_model(model_name, random_state=123, verbose=0, class_weight="balanced")

## Task Creation

We use Cyclops tasks to define our model's task (in this case, MortalityPrediction), train the model, make predictions, and evaluate performance. Cyclops task classes encapsulate the entire ML pipeline into a single, cohesive structure, making the process smooth and easy to manage.

In [ ]:
mortality_task = MortalityPredictionTask(
    {model_name: model}, task_features=features_list, task_target="outcome"
)

In [ ]:
mortality_task.list_models()

## Training

If `best_model_params` is passed to the `train` method, the best model will be selected after the hyperparameter search. The parameters in `best_model_params` indicate the values to create the parameters grid.

Note that the data preprocessor needs to be passed to the tasks methods if the Hugging Face dataset is not already preprocessed. 

In [ ]:
best_model_params = {
    "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    "learning_rate": ["constant", "optimal", "invscaling", "adaptive"],
    "eta0": [0.1, 0.01, 0.001, 0.0001],
    "metric": "roc_auc",
    "method": "grid",
}

mortality_task.train(
    dataset["train"],
    model_name=model_name,
    transforms=preprocessor,
    best_model_params=best_model_params,
)

We can add model parameters to the model card using the `log_model_parameters` method.

In [ ]:
report.log_model_parameters(
    params=mortality_task.list_models_params()[model_name],
)

In [ ]:
mortality_task.list_models_params()[model_name]

## Prediction

The prediction output can be either the whole Hugging Face dataset with the prediction columns added to it or the single column containing the predicted values.

In [ ]:
y_pred = mortality_task.predict(
    dataset["test"],
    model_name=model_name,
    transforms=preprocessor,
    proba=False,
    only_predictions=True,
)
len(y_pred)

## Evaluation

Evaluation is done using various evaluation metrics that provide different perspectives on the model's predictive abilities i.e. standard performance metrics and fairness metrics.

The standard performance metrics can be created using the `MetricCollection` object.

In [ ]:
metric_names = [
    "accuracy",
    "precision",
    "recall",
    "f1_score",
    "auroc",
    "roc_curve",
    "precision_recall_curve",
]
metrics = [create_metric(metric_name, task="binary") for metric_name in metric_names]
metric_collection = MetricCollection(metrics)

In addition to overall metrics, it might be interesting to see how the model performs on certain subpopulations. We can define these subpopulations using `SliceSpec` objects. 

In [ ]:
spec_list = [
    {
        "age": {
            "min_value": 30,
            "max_value": 50,
            "min_inclusive": True,
            "max_inclusive": False,
        }
    },
    {
        "age": {
            "min_value": 50,
            "max_value": 80,
            "min_inclusive": True,
            "max_inclusive": False,
        }
    },
    {"gender": {"value": 1}},
    {"gender": {"value": 0}},
]
slice_spec = SliceSpec(spec_list)

A `MetricCollection` can also be defined for the fairness metrics.

In [ ]:
specificity = create_metric(
    metric_name="specificity",
    task="binary",
)
sensitivity = create_metric(
    metric_name="sensitivity",
    task="binary",
)

fpr = 1 - specificity
fnr = 1 - sensitivity

ber = (fpr + fnr) / 2

fairness_metric_collection = MetricCollection(
    {
        "Sensitivity": sensitivity,
        "Specificity": specificity,
        "BER": ber,
    }
)

The FairnessConfig helps in setting up and evaluating the fairness of the model predictions.

In [ ]:
fairness_config = FairnessConfig(
    metrics=fairness_metric_collection,
    dataset=None,  # dataset is passed from the evaluator
    target_columns=None,  # target columns are passed from the evaluator
    groups=["gender", "age"],
    group_bins={"age": [50, 70]},
    group_base_values={"age": 20, "gender": 0},
    thresholds=[0.5],
)

The evaluate methods outputs the evaluation results and the Hugging Face dataset with the predictions added to it.

In [ ]:
results, dataset_with_preds = mortality_task.evaluate(
    dataset["test"],
    metric_collection,
    model_names=model_name,
    transforms=preprocessor,
    prediction_column_prefix="preds",
    slice_spec=slice_spec,
    batch_size=64,
    fairness_config=fairness_config,
    override_fairness_metrics=False,
)
dataset_with_preds

In [ ]:
results[model_name].keys()

In [ ]:
results[model_name]["overall"].keys()

In [ ]:
results["fairness"].keys()

In [ ]:
results["fairness"]["gender:0&age:(-inf - 50.0]"]

We can add a performance metric to the model card using the `log_performance_metric` method, which expects a dictionary where the keys are in the following format: `slice_name/metric_name`. For instance, `overall/accuracy`. 

We first need to process the evaluation results to get the metrics in the right format.

In [ ]:
# flatten the results to follow the model card schema
results_flat = {}
for name, model_results in results.items():
    results_flat[name] = {}
    for slice_name, slice_results in model_results.items():
        for metric_name, metric_value in slice_results.items():
            # remove the curve data
            if metric_name not in ["BinaryROCCurve", "BinaryPrecisionRecallCurve"]:
                results_flat[name][f"{slice_name}/{metric_name}"] = metric_value

In [ ]:
report.log_performance_metrics(results_flat[model_name])

We also use the `ClassificationPlotter` to plot the performance metrics and the add the figure to the model card using the `log_plotly_figure` method.

In [ ]:
plotter = ClassificationPlotter(task_type="binary", class_names=["0", "1"])
plotter.set_template("plotly_white")
plotter.set_colorway(
    [
        "#006ba6",
        "#ffbc42",
        "#0496ff",
        "#d81159",
        "#8f2d56",
        "#75dddd",
        "#508991",
        "#172a3a",
        "#004346",
        "#09bc8a",
    ]
)

In [ ]:
report.log_plotly_figure(
    fig=fig,
    alt="ROC Curve for Female Patients",
    section_name="quantitative analysis",
)

In [ ]:
# plotting the ROC curves for all the slices
roc_curves = {
    slice_name: slice_results["BinaryROCCurve"]
    for slice_name, slice_results in results[model_name].items()
}
aurocs = {
    slice_name: slice_results["BinaryAUROC"]
    for slice_name, slice_results in results[model_name].items()
}
roc_plot = plotter.roc_curve_comparison(roc_curves, aurocs=aurocs)
report.log_plotly_figure(
    fig=roc_plot,
    alt="ROC Curve for Female Patients",
    section_name="quantitative analysis",
)
roc_plot.show()

In [ ]:
# Plotting the overall classification metric values.
overall_performance = {
    metric_name: metric_value
    for metric_name, metric_value in results[model_name]["overall"].items()
    if metric_name not in ["BinaryROCCurve", "BinaryPrecisionRecallCurve"]
}
overall_performance_plot = plotter.metrics_value(
    overall_performance, title="Overall Performance"
)
report.log_plotly_figure(
    fig=overall_performance_plot,
    alt="Overall Performance",
    section_name="quantitative analysis",
)
overall_performance_plot.show()

In [ ]:
# Plotting the metric values for all the slices.
slice_metrics = {
    slice_name: {
        metric_name: metric_value
        for metric_name, metric_value in slice_results.items()
        if metric_name not in ["BinaryROCCurve", "BinaryPrecisionRecallCurve"]
    }
    for slice_name, slice_results in results[model_name].items()
}
slice_metrics_plot = plotter.metrics_comparison_bar(slice_metrics)
report.log_plotly_figure(
    fig=slice_metrics_plot,
    alt="Slice Metric Comparison",
    section_name="quantitative analysis",
)
slice_metrics_plot.show()

In [ ]:
# plotting the fairness metrics
fairness_results = copy.deepcopy(results["fairness"])
fairness_metrics = {}
# remove the group size from the fairness results and add it to the slice name
for slice_name, slice_results in fairness_results.items():
    group_size = slice_results.pop("Group Size")
    fairness_metrics[f"{slice_name} (Size={group_size})"] = slice_results

fairness_plot = plotter.metrics_comparison_bar(fairness_metrics)
report.log_plotly_figure(
    fig=fairness_plot,
    alt="Slice Metric Comparison",
    section_name="quantitative analysis",
)
fairness_plot.show()

## Report Generation

Before generating the model card, let us document some of the details of the model and some considerations involved in developing and using the model.


Let's start with populating the model details section, which includes the following fields by default:
- description: A high-level description of the model and its usage for a general audience.
- version: The version of the model.
- owners: The individuals or organizations that own the model.
- license: The license under which the model is made available.
- citation: The citation for the model.
- references: Links to resources that are relevant to the model.
- path: The path to where the model is stored.
- regulatory_requirements: The regulatory requirements that are relevant to the model.

We can add additional fields to the model details section by passing a dictionary to the `log_from_dict` method and specifying the section name as `model_details`. You can also use the `log_descriptor` method to add a new field object with a `description` attribute to any section of the model card.

In [ ]:
report.log_from_dict(
    data={
        "description": "The model was trained on a subset of the MIMIC-III dataset \
            to predict in-hospital mortality for patients admitted to the ICU with \
            heart failure. In-hospital mortality is defined as vital status at \
            hospital discharge.",
    },
    section_name="model_details",
)
report.log_version(
    version_str="0.0.1",
    date="2023-06-05",
    description="Initial Release",
)
report.log_owner(name="CyclOps Team", contact="vectorinstitute.github.io/cyclops/")
report.log_license(identifier="Apache-2.0")
report.log_reference(
    link="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html"  # noqa: E501
)

Next, let's populate the considerations section, which includes the following fields by default:
- users: The intended users of the model.
- use_cases: The use cases for the model. These could be primary, downstream or out-of-scope use cases.
- fairness_assessment: A description of the benefits and harms of the model for different groups as well as the steps taken to mitigate the harms.
- ethical_considerations: The risks associated with using the model and the steps taken to mitigate them. This can be populated using the  `log_risk` method.



In [ ]:
report.log_from_dict(
    data={
        "users": [
            {"description": "Hospitals"},
            {"description": "Clinicians"},
        ]
    },
    section_name="considerations",
)
report.log_user(description="ML Engineers")
report.log_use_case(
    description="Predicting mortality of patients admitted to the ICU with heart \
        failure.",
    kind="primary",
)
report.log_use_case(
    description="Predicting mortality patients admitted to other units of a hospital \
        with different conditions.",
    kind="out-of-scope",
)
report.log_fairness_assessment(
    affected_group="gendera, age",
    benefit="Improved health outcomes for patients.",
    harm="Biased predictions for patients in certain groups (e.g. older patients) \
        may lead to worse health outcomes.",
    mitigation_strategy="We will monitor the performance of the model on these groups \
        and retrain the model if the performance drops below a certain threshold.",
)
report.log_risk(
    risk="The model may be used to make decisions that affect the health of patients.",
    mitigation_strategy="The model should be continuously monitored for performance \
        and retrained if the performance drops below a certain threshold.",
)

Once the model card is populated, you can generate the report using the `export` method. The report is generated in the form of an HTML file. A JSON file containing the model card data will also be generated along with the HTML file. By default, the files will be saved in a folder named `cyclops_reports` in the current working directory. You can change the path by passing a `output_dir` argument when instantiating the `ModelCardReport` class.

In [ ]:
report.log_image(
    "/home/malinoori/cyclops/use_cases/plots/roc_curve_multilabel.png",
    alt="test",
    section_name="quantitative_analysis",
)

In [ ]:
report.export()